In [2]:
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import layers, models
import pickle
import tensorflow.keras as keras


2023-11-11 10:54:42.504062: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#Import
#train_df = tf.keras.utils.image_dataset_from_directory("archive/train",seed=45,image_size=(32, 32),batch_size=64)
#test_df = tf.keras.utils.image_dataset_from_directory("archive/test",seed=123,image_size=(32, 32),batch_size=64)

# directories
train_img_path = 'archive/train'
test_img_path = 'archive/test'

w,h = 32,32
batch_size = 32

train_data_gen = ImageDataGenerator(
    rescale = 1.0/255
)
test_data_gen = ImageDataGenerator(rescale = 1.0/255)

train_Dataset = train_data_gen.flow_from_directory(
    train_img_path,
    target_size = (w,h),
    batch_size = batch_size,
    class_mode = 'binary',
    shuffle = True
)

test_Dataset = test_data_gen.flow_from_directory(
    test_img_path,
    target_size = (w,h),
    batch_size = batch_size,
    class_mode = 'binary',
    shuffle = False
)


Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [16]:
# model = tf.keras.applications.NASNetLarge(
#     input_shape=None,
#     include_top=False,
#     weights="imagenet",
#     input_tensor=None,
#     pooling=None,
#     classes=1000,
#     classifier_activation="softmax",
# )


#model = keras.applications.NASNetMobile(weights=None, input_shape=(32, 32, 3), classes=7)
layers = []
layers.append(tf.keras.layers.Rescaling(1./255)) # Normalise pixel values
layers.append(tf.keras.layers.Conv2D(32, 3, activation='relu'))
layers.append(tf.keras.layers.MaxPooling2D())
layers.append(tf.keras.layers.Flatten())

# Building the ANN
layers.append(tf.keras.layers.Dense(64, activation='relu'))
layers.append(tf.keras.layers.Dense(32, activation='relu'))
layers.append(tf.keras.layers.Dense(32, activation='softmax'))


layers.append(tf.keras.layers.Dense(1, activation='sigmoid'))
layers.append(tf.keras.layers.Dense(1, activation='softmax'))


# Create and compile the model from layers
model = tf.keras.Sequential(layers)
model.compile(
  optimizer='Adamax',
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# Build the model so we can see a summary
model.build(input_shape=(None, 32, 32, 3))
model.summary()

# model = 

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 32, 32, 3)         0         
                                                                 
 conv2d_6 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 7200)              0         
                                                                 
 dense_6 (Dense)             (None, 64)                460864    
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                      

In [4]:

# Define the model
model2 = models.Sequential()

# Convolutional layers
#model2.add(layers.Rescaling(1./255))
model2.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Conv2D(128, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))

# Flatten layer
model2.add(layers.Flatten())

# Dense layers
model2.add(layers.Dense(256, activation='relu'))
model2.add(layers.Dropout(0.5))  # Optional dropout for regularization
model2.add(layers.Dense(1, activation='sigmoid'))  # Output layer with 1 neuron (binary classification)

# Compile the model
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print a summary of the model architecture
model2.summary()


2023-11-11 10:54:52.138002: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-11 10:54:52.138251: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 2, 2, 128)        0

In [30]:
model2.compile(
  optimizer='adam',
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

history = model2.fit(
  train_Dataset,
  validation_data=test_Dataset,
  epochs=20,
  verbose=1
)

Epoch 1/20


2023-11-11 12:00:31.928173: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-11-11 12:00:32.440804: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-11-11 12:00:32.532175: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.


3124/3125 [============================>.] - ETA: 0s - loss: 115414.1641 - accuracy: 0.6474 - precision_1: 0.6517 - recall_1: 0.6331

2023-11-11 12:01:20.253528: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-11-11 12:01:20.334190: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3125/3125 [==============================] - 55s 16ms/step - loss: 115426.5469 - accuracy: 0.6474 - precision_1: 0.6517 - recall_1: 0.6330 - val_loss: 393558.6562 - val_accuracy: 0.6097 - val_precision_1: 0.5798 - val_recall_1: 0.7970
Epoch 2/20
3125/3125 [==============================] - 48s 15ms/step - loss: 177916.3281 - accuracy: 0.6410 - precision_1: 0.6454 - recall_1: 0.6257 - val_loss: 936007.8125 - val_accuracy: 0.5955 - val_precision_1: 0.5636 - val_recall_1: 0.8458
Epoch 3/20
3125/3125 [==============================] - 48s 15ms/step - loss: 302175.2188 - accuracy: 0.6306 - precision_1: 0.6349 - recall_1: 0.6144 - val_loss: 1150979.7500 - val_accuracy: 0.5976 - val_precision_1: 0.5692 - val_recall_1: 0.8029
Epoch 4/20
3125/3125 [==============================] - 50s 16ms/step - loss: 422488.0625 - accuracy: 0.6340 - precision_1: 0.6384 - recall_1: 0.6182 - val_loss: 2106140.7500 - val_accuracy: 0.5750 - val_precision_1: 0.5592 - val_recall_1: 0.7087
Epoch 5/20
3125/3125 [===

KeyboardInterrupt: 

In [ ]:
history = model.fit(
  train_Dataset,
  validation_data=test_Dataset,
  epochs=10,
  verbose=1
)


Epoch 1/10


2023-11-11 10:27:49.088738: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [100000]
	 [[{{node Placeholder/_4}}]]
2023-11-11 10:27:49.089041: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [100000]
	 [[{{node Placeholder/_4}}]]
2023-11-11 10:27:49.783804: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-11-11 10:27:49.862417: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain term

1563/1563 [==============================] - ETA: 0s - loss: 0.4681 - accuracy: 0.5000 - precision_1: 0.5000 - recall_1: 1.0000

2023-11-11 10:28:21.180368: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [20000]
	 [[{{node Placeholder/_4}}]]
2023-11-11 10:28:21.180605: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [20000]
	 [[{{node Placeholder/_4}}]]
2023-11-11 10:28:21.264501: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1563/1563 [==============================] - 38s 21ms/step - loss: 0.4681 - accuracy: 0.5000 - precision_1: 0.5000 - recall_1: 1.0000 - val_loss: 0.3537 - val_accuracy: 0.5000 - val_precision_1: 0.5000 - val_recall_1: 1.0000
Epoch 2/10
1563/1563 [==============================] - 25s 16ms/step - loss: 0.3303 - accuracy: 0.5000 - precision_1: 0.5000 - recall_1: 1.0000 - val_loss: 0.3022 - val_accuracy: 0.5000 - val_precision_1: 0.5000 - val_recall_1: 1.0000
Epoch 3/10
1563/1563 [==============================] - 25s 16ms/step - loss: 0.3182 - accuracy: 0.5000 - precision_1: 0.5000 - recall_1: 1.0000 - val_loss: 0.3467 - val_accuracy: 0.5000 - val_precision_1: 0.5000 - val_recall_1: 1.0000
Epoch 4/10
1563/1563 [==============================] - 25s 16ms/step - loss: 0.3392 - accuracy: 0.5000 - precision_1: 0.5000 - recall_1: 1.0000 - val_loss: 0.3400 - val_accuracy: 0.5000 - val_precision_1: 0.5000 - val_recall_1: 1.0000
Epoch 5/10
1563/1563 [==============================] - 25s 16ms/st

In [16]:
model2.save('m2.h5')


In [27]:
#user_input = input("Enter your input: ")
user_input1 = "archive/train/REAL/0000 (5).jpg"
user_input = '/Users/brian/Desktop/IMG_6017.jpg'
#"sample_data/train_fake-1477_4.jpg"

In [29]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Preprocess the user input (assuming user_input is the input image file path)
img = image.load_img(user_input1, target_size=(32, 32))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
#img_array /= 255.0  # Normalize pixel values

# Get model predictions
predictions = model2.predict(img_array)
predicted_class = np.argmax(predictions)

binary_predictions = (predictions > 0.5).astype(int)
binary_predictions

print(train_Dataset.class_indices)

print("Model predictions:", binary_predictions)
#print(predicted_class)

# if predicted_class == 0:
#     predicted_label = "real"
# else:
#     predicted_label = "fake"
# a
#     print(predicted_label)

1/1 [==============================] - 0s 19ms/step
{'FAKE': 0, 'REAL': 1}
Model predictions: [[1]]


In [18]:
predictions = model2.predict('archive/test/FAKE/0 (4).jpg')
predictions

IndexError: tuple index out of range

In [23]:
threshold = 0.99

# Get model predictions
predictions = model.predict(img_array)

# Check if the prediction score is greater than or equal to the threshold
if predictions >= threshold:
    predicted_class = "fake"
else:
    predicted_class = "real"

print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 24ms/step
Predicted class: real


In [15]:
binary_predictions = (predictions > 0.5).astype(int)
binary_predictions

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [25]:
# model.build(input_shape=(None, 32, 32, 3))
# model.summary()


# history = model.fit(
#   train_df,
#   validation_data=test_df,
#   epochs=5,
#   verbose=1
# )